In [1]:
import numpy as np

# Reservoir Rock Properties

In [3]:
perm = 50
poro = 0.2
height = 100
re = 1000

# Fluid Properties

In [4]:
visc = 5.

# Well Conditions

In [5]:
rw = 0.25
qw = 5000

# Gridding

In [6]:
N = 4

In [7]:
gamma = (re/rw)**(1/N)

In [8]:
gamma

7.952707287670506

In [9]:
r1 = rw*np.log(gamma)/(1-1/gamma)

In [10]:
r1

0.5929358367900616

In [19]:
radii = r1*gamma**np.arange(-1,N+1)

In [26]:
volume = np.pi*(radii[2:]**2-2*radii[1:-1]**2+radii[:-2]**2)/(2*np.log(gamma))*height

In [27]:
volume

array([1.63160604e+03, 1.03191827e+05, 6.52642415e+06, 4.12767306e+08])

In [29]:
a = volume*poro/1

In [42]:
a

array([3.26321208e+02, 2.06383653e+04, 1.30528483e+06, 8.25534612e+07])

In [34]:
rrock = perm*height/np.log(gamma)*2*np.pi

In [35]:
phase = 1/(visc*1)

In [38]:
trans = phase*rrock/158

In [39]:
trans

19.178566485923028

# Constructing Matrices

In [53]:
T = np.zeros((N,N))
A = np.zeros((N,N))
Q = np.zeros((N,1))

In [54]:
for i in range(N):
  if i==0:
    Q[i] = -qw
    T[i,i] = trans
    T[i,i+1] = -trans
  elif i==N-1:
    T[i,i] = trans
    T[i,i-1] = -trans
  else:
    T[i,i] = 2*trans
    T[i,i-1] = -trans
    T[i,i+1] = -trans

  A[i,i] = a[i]

In [55]:
T

array([[ 19.17856649, -19.17856649,   0.        ,   0.        ],
       [-19.17856649,  38.35713297, -19.17856649,   0.        ],
       [  0.        , -19.17856649,  38.35713297, -19.17856649],
       [  0.        ,   0.        , -19.17856649,  19.17856649]])

In [56]:
A

array([[3.26321208e+02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.06383653e+04, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.30528483e+06, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 8.25534612e+07]])

In [57]:
Q

array([[-5000.],
       [    0.],
       [    0.],
       [    0.]])

# Time Iteration

In [65]:
P = np.array([3000]*4).reshape((-1,1))

In [66]:
P

array([[3000],
       [3000],
       [3000],
       [3000]])

In [67]:
for t in range(3):
  Act = A*1e-5
  LHS = T+Act
  RHS = np.matmul(Act,P)+Q
  P = np.linalg.solve(LHS,RHS)
  print(P.flatten())

[2328.71461413 2589.30809785 2845.48205966 2996.49178838]
[2257.09697614 2517.79249268 2777.71841823 2991.52470638]
[2225.64169554 2486.34404565 2746.70797395 2985.96632976]
